# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Read Dataset

In [3]:
df = pd.read_csv('../datasets/cleaned_dataset.csv')
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HDays,ADays,HASLxM,HASTLxM,HACLxM,HAFLxM,HAYCLxM,HARCLxM,HAGSLxM,HAGCLxM,HAGSLxHM,HAGCLxHM,HGDLxM,HGDLxHM,HAPLxM,HAPLxHM,HWPLxM,HWPLxHM,HWinStreak,HUPLxM,HUPLxHM,HUnbStreak,AASLxM,AASTLxM,AACLxM,AAFLxM,AAYCLxM,AARCLxM,AAGSLxM,AAGCLxM,AAGSLxAM,AAGCLxAM,AGDLxM,AGDLxAM,AAPLxM,AAPLxAM,AWPLxM,AWPLxAM,AWinStreak,AUPLxM,AUPLxAM,AUnbStreak,HDef,HMid,HAtt,HOvr,ADef,AMid,AAtt,AOvr,Venue
0,2012-08-18,arsenal,sunderland,0,0,14,3,4,2,12,8,7,0,0,0,0,0,1.40,4.5,8.5,D,10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,82,84,82.00,74,75,78,75.67,Emirates Stadium
1,2012-09-01,swansea,sunderland,2,2,14,4,10,3,11,7,8,0,1,1,1,0,2.20,3.3,3.4,D,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,73,73,73,73.00,74,75,77,75.33,Swansea.com Stadium
2,2012-09-22,west-ham,sunderland,1,1,20,4,11,2,12,10,10,1,2,1,0,0,2.10,3.3,3.6,D,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3.0,71,73,75,73.00,74,75,77,75.33,Boleyn Ground
3,2012-10-06,city,sunderland,3,0,26,8,16,6,10,10,15,4,0,3,0,0,1.29,5.5,11.0,W,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,5.0,81,83,85,83.00,74,75,77,75.33,Etihad Stadium
4,2012-10-27,stoke,sunderland,0,0,11,6,6,3,12,6,7,3,2,2,0,0,1.95,3.3,4.2,D,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,75,76,75,75.33,74,75,77,75.33,Bet365 Stadium


In [4]:
df.shape

(3800, 72)

In [5]:
# remove variables showing information that cannot be known before the start of the match
df = df[[
         'HDays', 'ADays',
         'HASLxM', 'HASTLxM', 
         'HACLxM', 'HAFLxM', 'HAYCLxM', 'HARCLxM',
         'HAGSLxM', 'HAGCLxM', 'HAGSLxHM', 'HAGCLxHM',
         'HGDLxM', 'HGDLxHM', 'HAPLxM', 'HAPLxHM',
         'HWPLxM', 'HWPLxHM',
         'HWinStreak',
         'HUPLxM', 'HUPLxHM', 
         'HUnbStreak',
         'AASLxM', 'AASTLxM', 
         'AACLxM', 'AAFLxM', 'AAYCLxM', 'AARCLxM',
         'AAGSLxM', 'AAGCLxM', 'AAGSLxAM', 'AAGCLxAM',
         'AGDLxM', 'AGDLxAM', 'AAPLxM','AAPLxAM',
         'AWPLxM', 'AWPLxAM', 
         'AWinStreak',
         'AUPLxM', 'AUPLxAM',
         'AUnbStreak',
         'HDef', 'HMid', 'HAtt', 'HOvr',
         'ADef', 'AMid', 'AAtt', 'AOvr',
         'B365H', 'B365D', 'B365A',
         'Venue',
         'Result'
        ]]

df.head()

,HDays,ADays,HASLxM,HASTLxM,HACLxM,HAFLxM,HAYCLxM,HARCLxM,HAGSLxM,HAGCLxM,HAGSLxHM,HAGCLxHM,HGDLxM,HGDLxHM,HAPLxM,HAPLxHM,HWPLxM,HWPLxHM,HWinStreak,HUPLxM,HUPLxHM,HUnbStreak,AASLxM,AASTLxM,AACLxM,AAFLxM,AAYCLxM,AARCLxM,AAGSLxM,AAGCLxM,AAGSLxAM,AAGCLxAM,AGDLxM,AGDLxAM,AAPLxM,AAPLxAM,AWPLxM,AWPLxAM,AWinStreak,AUPLxM,AUPLxAM,AUnbStreak,HDef,HMid,HAtt,HOvr,ADef,AMid,AAtt,AOvr,B365H,B365D,B365A,Venue,Result
0,10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,82,84,82.00,74,75,78,75.67,1.40,4.5,8.5,Emirates Stadium,D
1,7,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,73,73,73,73.00,74,75,77,75.33,2.20,3.3,3.4,Swansea.com Stadium,D
2,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3.0,71,73,75,73.00,74,75,77,75.33,2.10,3.3,3.6,Boleyn Ground,D
3,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,5.0,81,83,85,83.00,74,75,77,75.33,1.29,5.5,11.0,Etihad Stadium,W
4,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,75,76,75,75.33,74,75,77,75.33,1.95,3.3,4.2,Bet365 Stadium,D


In [6]:
df.shape

(3800, 55)

In [7]:
df = df.dropna(axis=0).reset_index(drop=True)
df.head()

,HDays,ADays,HASLxM,HASTLxM,HACLxM,HAFLxM,HAYCLxM,HARCLxM,HAGSLxM,HAGCLxM,HAGSLxHM,HAGCLxHM,HGDLxM,HGDLxHM,HAPLxM,HAPLxHM,HWPLxM,HWPLxHM,HWinStreak,HUPLxM,HUPLxHM,HUnbStreak,AASLxM,AASTLxM,AACLxM,AAFLxM,AAYCLxM,AARCLxM,AAGSLxM,AAGCLxM,AAGSLxAM,AAGCLxAM,AGDLxM,AGDLxAM,AAPLxM,AAPLxAM,AWPLxM,AWPLxAM,AWinStreak,AUPLxM,AUPLxAM,AUnbStreak,HDef,HMid,HAtt,HOvr,ADef,AMid,AAtt,AOvr,B365H,B365D,B365A,Venue,Result
0,3,4,18.2,10.4,8.2,9.6,1.3,0.0,1.8,1.1,1.4,1.0,7.0,4.0,1.7,1.5,50.0,40.0,1.0,70.0,70.0,1.0,13.4,8.4,5.9,11.8,1.6,0.0,1.4,1.5,1.0,1.4,-1.0,-4.0,1.3,1.0,40.0,20.0,0.0,50.0,60.0,0.0,79,80,79,79.33,74,75,77,75.33,1.40,4.5,8.5,Anfield,W
1,7,7,11.7,6.0,5.1,11.4,1.5,0.1,1.1,1.9,1.3,2.0,-8.0,-7.0,0.8,0.9,20.0,20.0,0.0,40.0,50.0,1.0,12.7,8.2,5.7,10.8,1.2,0.0,1.2,1.3,1.0,1.7,-1.0,-7.0,1.3,0.9,40.0,20.0,1.0,50.0,50.0,1.0,74,70,72,72.00,74,75,77,75.33,2.15,3.4,3.8,DW Stadium,L
2,3,4,6.2,3.2,3.9,10.8,1.5,0.0,1.1,1.8,1.8,1.9,-7.0,-1.0,1.1,1.4,30.0,30.0,0.0,50.0,80.0,3.0,11.2,7.5,4.9,10.9,1.2,0.0,1.4,1.3,1.1,1.7,1.0,-6.0,1.6,1.1,50.0,30.0,0.0,60.0,50.0,3.0,70,70,73,71.00,74,75,77,75.33,2.50,3.4,3.0,Madejski Stadium,W
3,10,10,12.4,6.8,6.0,11.0,1.5,0.1,1.2,1.4,1.3,1.2,-2.0,1.0,1.1,1.4,30.0,40.0,1.0,50.0,60.0,1.0,10.4,6.7,5.1,11.1,1.5,0.0,1.1,1.3,1.1,1.8,-2.0,-7.0,1.3,1.0,40.0,30.0,0.0,50.0,40.0,0.0,71,75,74,73.33,74,75,77,75.33,2.05,3.5,4.0,The Hawthorns,W
4,7,7,10.4,5.3,4.0,9.7,1.5,0.0,0.6,1.3,0.6,1.3,-7.0,-7.0,1.0,0.8,20.0,10.0,1.0,60.0,60.0,1.0,10.3,6.3,4.9,10.4,1.4,0.0,1.2,1.4,1.2,1.7,-2.0,-5.0,1.1,1.0,30.0,30.0,0.0,50.0,40.0,1.0,74,74,79,75.67,74,75,77,75.33,2.25,3.4,3.5,Loftus Road,W


In [8]:
df.shape

(3324, 55)

# 3. Splitting to Train and Test Set

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

## 3.1 Data Split 1

- Target Variable: Draw & Non-Draw
- Sampling: Stratified

In [10]:
draw_df = df[df['Result'] == 'D']
non_draw_df = df[(df['Result'] == 'W') | (df['Result'] == 'L')]
non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})

print(draw_df.Result.value_counts())
print(non_draw_df.Result.value_counts())

D    775
Name: Result, dtype: int64
ND    2549
Name: Result, dtype: int64


C:\Users\choibs\AppData\Local\Temp\ipykernel_6816\2887461011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})


In [11]:
# Setting up 5 fold_df
splits = [pd.DataFrame(columns=list(df.columns)) for i in range(5)]

In [12]:
# Setting up draw_df
draw_df = draw_df.sample(frac = 1, random_state=0)

split_sizes = [155,155,155,155,155]

for i in range(5):
    # copy section of draw_df to fold_df
    splits[i] = pd.concat([splits[i], draw_df.iloc[:split_sizes[i], :]])
    draw_df = draw_df.drop(draw_df.index[:split_sizes[i]])

In [13]:
# Setting up non_draw_df
non_draw_df = non_draw_df.sample(frac=1, random_state=0)

split_sizes = [509,510,510,510,510]

for i in range(5):
    # copy section of non_draw_df to fold_df
    splits[i] = pd.concat([splits[i], non_draw_df.iloc[:split_sizes[i], :]])
    non_draw_df = non_draw_df.drop(non_draw_df.index[:split_sizes[i]])

In [14]:
# save to csv files
for i in range(5):
    splits[i].to_csv(f'data-split-1/fold-{i+1}/test.csv', index=False)
    
    train_df = pd.DataFrame(columns=list(df.columns))
    for j in range(5):
        if (j != i):
            train_df = pd.concat([train_df, splits[j]])

    train_df.to_csv(f'data-split-1/fold-{i+1}/train.csv', index=False)

## 3.2 Data Split 2

- Target Variable: Draw & Non-Draw
- Sampling: Balanced

In [15]:
draw_df = df[df['Result'] == 'D']
non_draw_df = df[(df['Result'] == 'W') | (df['Result'] == 'L')]
non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})

print(draw_df.Result.value_counts())
print(non_draw_df.Result.value_counts())

D    775
Name: Result, dtype: int64
ND    2549
Name: Result, dtype: int64


C:\Users\choibs\AppData\Local\Temp\ipykernel_6816\2887461011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_draw_df['Result'] = non_draw_df['Result'].map({"W": "ND", "L": "ND"})


In [16]:
d_range = [(0, 620), (40, 660), (79, 699), (118, 738), (155, 775)]
nd_range = [(0, 620), (482, 1102), (964, 1584), (1447, 2067), (1929, 2549)]

for i in range(5):
    test_df = pd.DataFrame(columns=list(df.columns))
    train_df = pd.DataFrame(columns=list(df.columns))
    
    # prepare d_df and nd_df
    d_df = draw_df.sample(frac = 1, random_state=0)
    nd_df = non_draw_df.sample(frac=1, random_state=0)
    
    # copy section of d_df to train set
    d_start = d_range[i][0]
    d_end = d_range[i][1]
    train_df = pd.concat([train_df, d_df.iloc[d_start:d_end, :]])
    d_df = d_df.drop(d_df.index[d_start:d_end])
    
    # copy section of nd_df to train set
    nd_start = nd_range[i][0]
    nd_end = nd_range[i][1]
    train_df = pd.concat([train_df, nd_df.iloc[nd_start:nd_end, :]])
    nd_df = nd_df.drop(nd_df.index[nd_start:nd_end])
    
    # prepare test set
    test_df = pd.concat([test_df, d_df])
    test_df = pd.concat([test_df, nd_df])
    
    # save to csv files
    train_df.to_csv(f'data-split-2/fold-{i+1}/train.csv', index=False)
    test_df.to_csv(f'data-split-2/fold-{i+1}/test.csv', index=False)

## 3.3 Data Split 3

- Target Variable: Win, Draw, Lose
- Sampling: Stratified

In [17]:
win_df = df[df['Result'] == 'W']
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(win_df.Result.value_counts())
print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

W    1490
Name: Result, dtype: int64
D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [18]:
# Setting up 5 fold_df
splits = [pd.DataFrame(columns=list(df.columns)) for i in range(5)]

In [19]:
# Setting up win_df
win_df = win_df.sample(frac = 1, random_state=0)

split_sizes = [298,298,298,298,298]

for i in range(5):
    # copy section of win_df to fold_df
    splits[i] = pd.concat([splits[i], win_df.iloc[:split_sizes[i], :]])
    win_df = win_df.drop(win_df.index[:split_sizes[i]])

In [20]:
# Setting up draw_df
draw_df = draw_df.sample(frac = 1, random_state=0)

split_sizes = [155,155,155,155,155]

for i in range(5):
    # copy section of draw_df to fold_df
    splits[i] = pd.concat([splits[i], draw_df.iloc[:split_sizes[i], :]])
    draw_df = draw_df.drop(draw_df.index[:split_sizes[i]])

In [21]:
# Setting up lose_df
lose_df = lose_df.sample(frac = 1, random_state=0)

split_sizes = [211,212,212,212,212]

for i in range(5):
    # copy section of lose_df to fold_df
    splits[i] = pd.concat([splits[i], lose_df.iloc[:split_sizes[i], :]])
    lose_df = lose_df.drop(lose_df.index[:split_sizes[i]])

In [22]:
# save to csv files
for i in range(5):
    splits[i].to_csv(f'data-split-3/fold-{i+1}/test.csv', index=False)
    
    train_df = pd.DataFrame(columns=list(df.columns))
    for j in range(5):
        if (j != i):
            train_df = pd.concat([train_df, splits[j]])

    train_df.to_csv(f'data-split-3/fold-{i+1}/train.csv', index=False)

## 3.4 Data Split 4

- Target Variable: Win, Draw, Lose
- Sampling: Balanced

In [23]:
win_df = df[df['Result'] == 'W']
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(win_df.Result.value_counts())
print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

W    1490
Name: Result, dtype: int64
D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [24]:
w_range = [(0, 620), (217, 837), (435, 1055), (655, 1275), (870, 1490)]
d_range = [(0, 620), (40, 660), (79, 699), (118, 738), (155, 775)]
l_range = [(0, 620), (111, 731), (220, 840), (330, 950), (439, 1059)]

for i in range(5):
    test_df = pd.DataFrame(columns=list(df.columns))
    train_df = pd.DataFrame(columns=list(df.columns))
    
    # prepare d_df and nd_df
    w_df = win_df.sample(frac = 1, random_state=0)
    d_df = draw_df.sample(frac=1, random_state=0)
    l_df = lose_df.sample(frac=1, random_state=0)
    
    # copy section of w_df to train set
    w_start = w_range[i][0]
    w_end = w_range[i][1]
    train_df = pd.concat([train_df, w_df.iloc[w_start:w_end, :]])
    w_df = w_df.drop(w_df.index[w_start:w_end])
    
    # copy section of d_df to train set
    d_start = d_range[i][0]
    d_end = d_range[i][1]
    train_df = pd.concat([train_df, d_df.iloc[d_start:d_end, :]])
    d_df = d_df.drop(d_df.index[d_start:d_end])
    
    # copy section of l_df to train set
    l_start = l_range[i][0]
    l_end = l_range[i][1]
    train_df = pd.concat([train_df, l_df.iloc[l_start:l_end, :]])
    l_df = l_df.drop(l_df.index[l_start:l_end])
    
    # prepare test set
    test_df = pd.concat([test_df, w_df])
    test_df = pd.concat([test_df, d_df])
    test_df = pd.concat([test_df, l_df])
    
    # save to csv files
    train_df.to_csv(f'data-split-4/fold-{i+1}/train.csv', index=False)
    test_df.to_csv(f'data-split-4/fold-{i+1}/test.csv', index=False)

## 3.5 Data Split 5

- Target Variable: Draw, Lose
- Sampling: Stratified

In [25]:
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [26]:
# Setting up 5 fold_df
splits = [pd.DataFrame(columns=list(df.columns)) for i in range(5)]

In [27]:
# Setting up draw_df
draw_df = draw_df.sample(frac = 1, random_state=0)

split_sizes = [155,155,155,155,155]

for i in range(5):
    # copy section of draw_df to fold_df
    splits[i] = pd.concat([splits[i], draw_df.iloc[:split_sizes[i], :]])
    draw_df = draw_df.drop(draw_df.index[:split_sizes[i]])

In [28]:
# Setting up lose_df
lose_df = lose_df.sample(frac = 1, random_state=0)

split_sizes = [211,212,212,212,212]

for i in range(5):
    # copy section of lose_df to fold_df
    splits[i] = pd.concat([splits[i], lose_df.iloc[:split_sizes[i], :]])
    lose_df = lose_df.drop(lose_df.index[:split_sizes[i]])

In [29]:
# save to csv files
for i in range(5):
    splits[i].to_csv(f'data-split-5/fold-{i+1}/test.csv', index=False)
    
    train_df = pd.DataFrame(columns=list(df.columns))
    for j in range(5):
        if (j != i):
            train_df = pd.concat([train_df, splits[j]])

    train_df.to_csv(f'data-split-5/fold-{i+1}/train.csv', index=False)

## 3.6 Data Split 6

- Target Variable: Draw, Lose
- Sampling: Balanced

In [30]:
draw_df = df[df['Result'] == 'D']
lose_df = df[df['Result'] == 'L']

print(draw_df.Result.value_counts())
print(lose_df.Result.value_counts())

D    775
Name: Result, dtype: int64
L    1059
Name: Result, dtype: int64


In [31]:
d_range = [(0, 620), (40, 660), (79, 699), (118, 738), (155, 775)]
l_range = [(0, 620), (111, 731), (220, 840), (330, 950), (439, 1059)]

for i in range(5):
    test_df = pd.DataFrame(columns=list(df.columns))
    train_df = pd.DataFrame(columns=list(df.columns))
    
    # prepare d_df and nd_df
    d_df = draw_df.sample(frac=1, random_state=0)
    l_df = lose_df.sample(frac=1, random_state=0)
    
    # copy section of d_df to train set
    d_start = d_range[i][0]
    d_end = d_range[i][1]
    train_df = pd.concat([train_df, d_df.iloc[d_start:d_end, :]])
    d_df = d_df.drop(d_df.index[d_start:d_end])
    
    # copy section of l_df to train set
    l_start = l_range[i][0]
    l_end = l_range[i][1]
    train_df = pd.concat([train_df, l_df.iloc[l_start:l_end, :]])
    l_df = l_df.drop(l_df.index[l_start:l_end])
    
    # prepare test set
    test_df = pd.concat([test_df, d_df])
    test_df = pd.concat([test_df, l_df])
    
    # save to csv files
    train_df.to_csv(f'data-split-6/fold-{i+1}/train.csv', index=False)
    test_df.to_csv(f'data-split-6/fold-{i+1}/test.csv', index=False)